In [3]:
import os

In [1]:
%pwd

'/Users/raviraja/End-to-End-Machine-Learning-Project-with-MLflow/research'

In [4]:
os.chdir("../")

In [5]:
%pwd

'/Users/raviraja/End-to-End-Machine-Learning-Project-with-MLflow'

In [22]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [23]:
from ml_project.constants import *
from ml_project.utils.common import read_yaml, create_directories

In [24]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)
        
        create_directories([self.config.artifacts_root])
    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        create_directories([config.root_dir])
        
        data_ingestion_config = DataIngestionConfig(
            root_dir = Path(config.root_dir),
            source_URL = config.source_URL,
            local_data_file = Path(config.local_data_file),
            unzip_dir = Path(config.unzip_dir)
        )
        
        return data_ingestion_config

In [25]:
# components

import os
import urllib.request as request
import zipfile
from ml_project import logger
from ml_project.utils.common import get_size

In [28]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config
        
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"Downloaded file: {filename} with following info: \n{headers}")
        else:
            logger.info(f"File already exists: {get_size(Path(self.config.local_data_file))}")
        
    def extract_zip_file(self):
        """ 
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [29]:
# pipeline

try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e


[2025-03-10 11:00:51,091: INFO: common: yaml file: config/config.yaml loaded successfully]
[2025-03-10 11:00:51,094: INFO: common: yaml file: params.yaml loaded successfully]
[2025-03-10 11:00:51,094: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-03-10 11:00:51,095: INFO: common: created directory at: artifacts]
[2025-03-10 11:00:51,095: INFO: common: created directory at: artifacts/data_ingestion]
[2025-03-10 11:00:51,816: INFO: 334172574: Downloaded file: artifacts/data_ingestion/data.zip with following info: 
Connection: close
Content-Length: 23329
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "c69888a4ae59bc5a893392785a938ccd4937981c06ba8a9d6a21aa52b4ab5b6e"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: A983:340C96:23B0A6:64A113:67CE74BD
Accept-Ranges: bytes
Date: M